In [104]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains 
from selenium.webdriver import ActionChains 
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import requests

url='https://immo.vlan.be/fr'
print('ready')
source_pages = []  # contains the hmtl
source_pages_url = []
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(url)
time.sleep(7)
elem = driver.find_element_by_xpath("//input[@placeholder='Où ? Ville, Code Postal, Province ou Région.']")
elem.clear()
elem.send_keys("7000")
elem.send_keys(Keys.RETURN)
time.sleep(7)
xpaths = []
xpath = "//h2[@class='card-title text-ellipsis mb-1 d-inline with-small-icon house']"
elements = driver.find_elements_by_xpath(xpath)
print(elements)
for i in range(len(elements)):  #change range to len(elements) for full scrap of the house in the page
    elem = elements[i]
    actions = ActionChains(driver)
    actions.move_to_element(elem)
    actions.key_down(Keys.LEFT_CONTROL)
    actions.click(elem)
    actions.perform()
    time.sleep(5)
    driver.switch_to_window(driver.window_handles[1])
    source_pages.append(driver.page_source)
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
 
 
driver.close()






ready
[<selenium.webdriver.remote.webelement.WebElement (session="b30ed7c74a0a30558fd7e6e448fa24f5", element="cbbe4c41-8a31-4157-9d05-b74d7d0e9c9d")>, <selenium.webdriver.remote.webelement.WebElement (session="b30ed7c74a0a30558fd7e6e448fa24f5", element="47176d43-d8a6-4265-9fd6-5f71950c5cbc")>, <selenium.webdriver.remote.webelement.WebElement (session="b30ed7c74a0a30558fd7e6e448fa24f5", element="0edc919c-200c-4479-ad8a-a519bb14aa93")>, <selenium.webdriver.remote.webelement.WebElement (session="b30ed7c74a0a30558fd7e6e448fa24f5", element="7c623506-ae5f-47e1-9f9f-176df8ef5a09")>, <selenium.webdriver.remote.webelement.WebElement (session="b30ed7c74a0a30558fd7e6e448fa24f5", element="a35d5a8b-3212-4e62-a8aa-e599338d27dc")>, <selenium.webdriver.remote.webelement.WebElement (session="b30ed7c74a0a30558fd7e6e448fa24f5", element="ba95ab38-496d-49c5-9a7c-953a0c876712")>, <selenium.webdriver.remote.webelement.WebElement (session="b30ed7c74a0a30558fd7e6e448fa24f5", element="1f02057e-0b0b-4b32-9b5e-83

C:\Users\jeremy\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: use driver.switch_to.window instead
C:\Users\jeremy\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: use driver.switch_to.window instead


In [121]:
for i in range(len(elements)):
    soup = BeautifulSoup(source_pages[i], 'lxml')
    
    adid = soup.find("meta", {"name": "cXenseParse:rob-immo-property-adid"})
    if adid != None:
        adid = adid.get("content")
    else:
        adid = None
        
    bedrooms = soup.find("meta", {"name": "cXenseParse:rob-immo-property-bedrooms"})
    if bedrooms!= None:
        bedrooms = bedrooms.get("content")
    else:
        bedrooms = None
        
    city = soup.find("meta", {"name": "cXenseParse:rob-immo-property-city"})
    if city != None:
        city = city.get("content")
    else:
        city = None
        
    price = soup.find("meta", {"name": "cXenseParse:rob-immo-property-price"})
    if price != None:
        price = price.get("content")
    else:
        price = None
        
    transaction_type = soup.find("meta", {"name": "cXenseParse:rob-immo-transaction-type"})
    if transaction_type != None:
        transaction_type = transaction_type.get("content")
    else:
        transaction_type = None
        
    subtype = soup.find("meta", {"name": "cXenseParse:rob-immo-property-subtype"})
    if subtype!= None:
        subtype = subtype.get("content")
    else:
        subtype = None
        
    zipcode = soup.find("meta", {"name": "cXenseParse:rob-immo-property-zipcode"})
    if zipcode != None:
        zipcode = zipcode.get("content")
    else:
        zipcode = None
        
    adresse = soup.find("span", {"class": "address-line"})
    if adresse != None:
        adresse = adresse.text
    else:
        adresse = None
        
    surface = soup.find("div", {"title": "Surface habitable"})
    if surface!= None:
        surface = surface.find("div", {"class":"fs-4"}).text
    else:
        surface = None
    
    exterieur = soup.find("div", {"id":"collapse_outdoor_details"}).find_all("div")
    facades, surface_terrain, garden, garden_surface, terrasse, terrasse_surface, piscine = None, None, None, None, None, None, None
    for element in exterieur:
        text = element.text.strip()
        if text == "Nombre de façades":
            facades = element.find_next_sibling().text.strip()
        elif text == "Surface du terrain (m²)":
            surface_terrain = element.find_next_sibling().text.strip()[:-3]
        elif text == "Jardin":
            garden = element.find_next_sibling().text.strip()
        elif text == "Surface du jardin":
            garden_surface = element.find_next_sibling().text.strip()[:-3]
        elif text == "Terrasse aménagée":
            terrasse = element.find_next_sibling().text.strip()
        elif text == "Surface de la terrasse":
            terrasse_surface = element.find_next_sibling().text.strip()[:-3]
        elif text == "Piscine extérieure":
            piscine = element.find_next_sibling().text.strip()

            
    interieur = soup.find("div", {"id":"collapse_indoor_details"}).find_all("div")
    nombre_chambres, garage, surface_habitable, meuble = None, None, None, None
    for element in interieur:
        text = element.text.strip()
        if text == "Nombre de chambres à coucher":
            nombre_chambres = element.find_next_sibling().text.strip()
        elif text == "Surface habitable":
            surface_habitable = element.find_next_sibling().text.strip()[:-3]
        elif text == "Garage":
            garage = element.find_next_sibling().text.strip()
        elif text == "Meublé":
            meuble = element.find_next_sibling().text.strip()
    
    sanitaires = soup.find("div", {"id":"collapse_kitchenbath_details"})
    salles_de_bain, toilettes = None, None
    if sanitaires != None:
        sanitaires = sanitaires.find_all("div")
        for element in sanitaires:
            text = element.text.strip()
            if text == "Nombre de salles de bain":
                salles_de_bain = element.find_next_sibling().text.strip()
            elif text == "Nombre de toilettes":
                toilettes = element.find_next_sibling().text.strip()
    
    equipments = soup.find("div", {"id":"collapse_equipment_details"})
    cheminee = None
    if equipments != None:
        equipments = equipments.find_all("div")
        for element in equipments:
            text = element.text.strip()
            if text == "Feu ouvert":
                cheminee = element.find_next_sibling().text.strip()
            elif text == "Nombre de toilettes":
                toilettes = element.find_next_sibling().text.strip()
    
    """print("id:", adid, "- chambres:", bedrooms, "- ville:", city, "- prix: ", price, "- type de vente:", transaction_type,
          "- sous-type:", subtype, "- code-postal", zipcode, "- adresse:",  adresse, "surface:", surface)
    
    print("- facades:", facades, "- surface_terrain:", surface_terrain,
          "- garden:", garden, "- garden_surface:", garden_surface, "- terrasse:", terrasse)
    print("interieur", nombre_chambres, garage, surface_habitable)"""